In [1]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/ACCESS_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import dt

import dask
import warnings
from dask.distributed import Client

In [2]:
plt.style.use('default')
warnings.filterwarnings('ignore')